# Visual Data Analysis of Fraudulent Transactions

Your CFO has also requested detailed trends data on specific card holders. Use the starter notebook to query your database and generate visualizations that supply the requested information as follows, then add your visualizations and observations to your markdown report.

In [31]:
# Initial imports
import pandas as pd
import calendar
import hvplot.pandas
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

In [32]:
# Load environment variables from .env file
load_dotenv()

username = os.getenv('USERNAME')
password = os.getenv('PASSWORD')
server = os.getenv('SERVER')
port = os.getenv('PORT')
db_name = os.getenv('DB_NAME')

# Database url
db_url = f'postgresql://{username}:{password}@{server}:{port}/{db_name}'

# Create a connection to the database
engine = create_engine(db_url)

## Data Analysis Question 1
The CFO of your firm has requested a report to help analyze potential fraudulent transactions. Using your newly created database, generate queries that will discover the information needed to answer the following questions, then use your repository's ReadME file to create a markdown report you can share with the CFO:

* Some fraudsters hack a credit card by making several small transactions (generally less than $2.00), which are typically ignored by cardholders. 

  * How can you isolate (or group) the transactions of each cardholder?

  * Count the transactions that are less than $2.00 per cardholder. 
  
  * Is there any evidence to suggest that a credit card has been hacked? Explain your rationale.

In [33]:
# Write function that groups the transactions of each cardholder
query = """
    SELECT  
        ch.name AS cardholder_name,
        t.id AS transaction_id,
        t.date AS transaction_date,
        t.amount AS transaction_amount 
    FROM card_holder ch 
    JOIN credit_card cc ON ch.id = cc.cardholder_id
    JOIN transaction t ON cc.card = t.card
    GROUP BY ch.name, t.id, t.date, t.amount
    ORDER BY ch.name; 
"""
# Convert query results to a DataFrame
transaction_by_cardholder_df = pd.read_sql(query, engine)

# View DataFrame
transaction_by_cardholder_df

,cardholder_name,transaction_id,transaction_date,transaction_amount
0,Austin Johnson,18,2018-11-23 17:40:55,7.61
1,Austin Johnson,24,2018-09-25 12:45:03,5.99
2,Austin Johnson,144,2018-02-27 12:53:43,11.36
3,Austin Johnson,196,2018-07-05 20:28:22,5.59
4,Austin Johnson,204,2018-03-26 08:14:09,6.20
...,...,...,...,...
3495,Stephanie Dalton,3398,2018-02-21 05:56:41,2.88
3496,Stephanie Dalton,3445,2018-01-13 22:18:00,2.11
3497,Stephanie Dalton,3452,2018-01-27 08:24:11,3.08
3498,Stephanie Dalton,3480,2018-01-08 21:44:34,1.68


In [34]:
# Count the transactions that are less than $2.00 per cardholder
transaction_count = transaction_by_cardholder_df[transaction_by_cardholder_df['transaction_amount'] < 2.00].groupby('cardholder_name')['transaction_amount'].count()

transaction_count

cardholder_name
Austin Johnson        7
Beth Hernandez        6
Brandon Pineda       21
Crystal Clark        19
Dana Washington       4
Danielle Green       16
Elizabeth Sawyer      3
Gary Jacobs           9
John Martin          19
Kevin Spencer        18
Kyle Tucker          12
Laurie Gibbs          3
Malik Carlson        19
Mark Lewis           16
Matthew Gutierrez    20
Megan Price          26
Michael Carroll       4
Michael Floyd        15
Nancy Contreras      16
Peter Mckay          22
Robert Johnson       10
Sara Cooper          14
Sean Taylor          18
Shane Shaffer        11
Stephanie Dalton     22
Name: transaction_amount, dtype: int64

A number of transactions less than $2.00 generally means there is fraudulent activity on a credit card. There are many cardholders that fit the condition and therefore we can assume that their credit card has been hacked. 

## Data Analysis Question 2
Take your investigation a step futher by considering the time period in which potentially fraudulent transactions are made. 

  * What are the top 100 highest transactions made between 7:00 am and 9:00 am?

  * Do you see any anomalous transactions that could be fraudulent?

  * Is there a higher number of fraudulent transactions made during this time frame versus the rest of the day?

  * If you answered yes to the previous question, explain why you think there might be fraudulent transactions during this time frame.

* What are the top 5 merchants prone to being hacked using small transactions?

* Create a view for each of your queries.

In [35]:
# Filter the transaction_by_cardholder_df dataframe for transactions made between 7:00am and 9:00 am
top_100_transactions = transaction_by_cardholder_df[(transaction_by_cardholder_df['transaction_date'].dt.hour >= 7) & (transaction_by_cardholder_df['transaction_date'].dt.hour <= 9)].nlargest(100, 'transaction_amount')

print(top_100_transactions)

        cardholder_name  transaction_id    transaction_date  \
2948     Robert Johnson            3163 2018-12-07 07:22:03   
546       Crystal Clark            2451 2018-03-05 08:26:08   
2596    Nancy Contreras            2840 2018-03-06 07:18:09   
3441   Stephanie Dalton            2461 2018-12-21 09:56:32   
488       Crystal Clark            1442 2018-01-22 08:07:03   
...                 ...             ...                 ...   
669     Dana Washington            2511 2018-07-11 09:57:31   
2277        Megan Price            3053 2018-04-07 07:18:20   
940         Gary Jacobs            2858 2018-07-16 09:25:54   
2073  Matthew Gutierrez            2998 2018-10-17 09:49:28   
3201        Sean Taylor            2819 2018-08-02 07:13:49   

      transaction_amount  
2948             1894.00  
546              1617.00  
2596             1334.00  
3441             1301.00  
488              1131.00  
...                  ...  
669                14.80  
2277               14.80  


In [36]:
less_two_dollar_transactions = top_100_transactions[top_100_transactions["transaction_amount"]<2].count()
less_two_dollar_transactions

cardholder_name       0
transaction_id        0
transaction_date      0
transaction_amount    0
dtype: int64

There are no fraudulent transaction between 7am - 9am that satisfy the leass than $2 condition. This leads us to assume that most fradulent activity happen outside of this window. 

In [37]:
# Write function that groups the top 5 merchants prone to being hacked using small transactions
query = """
    SELECT  
        m.name AS merchant_name,
        COUNT(*) AS transaction_count
    FROM merchant m 
    JOIN transaction t ON m.id = t.id_merchant
    WHERE t.amount < 2
    GROUP BY m.name
    ORDER BY transaction_count DESC
    LIMIT 5; 
"""

# Convert query results to a DataFrame
transaction_by_merchant_df = pd.read_sql(query, engine)

# View DataFrame
transaction_by_merchant_df

,merchant_name,transaction_count
0,Wood-Ramirez,7
1,Hood-Phillips,6
2,Baker Inc,6
3,"Mcdaniel, Hines and Mcfarland",5
4,Hamilton-Mcfarland,5


## Data Analysis Question 3

The two most important customers of the firm may have been hacked. Verify if there are any fraudulent transactions in their history. For privacy reasons, you only know that their cardholder IDs are 2 and 18.

* Using hvPlot, create a line plot representing the time series of transactions over the course of the year for each cardholder separately. 

* Next, to better compare their patterns, create a single line plot that containins both card holders' trend data.  

* What difference do you observe between the consumption patterns? Does the difference suggest a fraudulent transaction? Explain your rationale in the markdown report.

In [ ]:
# loading data for card holder 2 and 18 from the database
# Write the query
query = """
        SELECT  
                ch.id AS cardholder_id,
                t.id AS transaction_id,
                t.date AS transaction_date,
                t.amount AS transaction_amount 
        FROM card_holder ch 
        JOIN credit_card cc ON ch.id = cc.cardholder_id
        JOIN transaction t ON cc.card = t.card
        WHERE cc.cardholder_id IN (2, 18)
        ORDER BY ch.id, t.date
        """
# Create a DataFrame from the query result. HINT: Use pd.read_sql(query, engine)
important_customer_df = pd.read_sql(query, engine)

# View DataFrame
important_customer_df


,cardholder_id,transaction_id,transaction_date,transaction_amount
0,2,2439,2018-01-06 02:16:41,1.33
1,2,1867,2018-01-06 05:13:20,10.82
2,2,3028,2018-01-07 15:10:27,17.29
3,2,998,2018-01-10 10:07:20,10.91
4,2,2655,2018-01-16 06:29:35,17.64
...,...,...,...,...
227,18,2148,2018-12-23 03:33:56,4.36
228,18,1994,2018-12-27 18:46:57,1.70
229,18,114,2018-12-28 08:45:26,3.46
230,18,1228,2018-12-28 09:00:45,12.88


In [ ]:
# Plot for cardholder 2
cardholder_two = important_customer_df[important_customer_df['cardholder_id']==2]
cardholder_two_plot = cardholder_two.hvplot.line(
    x='transaction_date',
    y='transaction_amount',
    xlabel='Transaction Date',
    ylabel='Transaction Amount',
    title = 'Time Series of Transactions for Cardholder 2'
)

cardholder_two_plot

:Curve   [transaction_date]   (transaction_amount)

In [ ]:
# Plot for cardholder 18
cardholder_eighteen = important_customer_df[important_customer_df['cardholder_id']==18]
cardholder_eighteen_plot = cardholder_eighteen.hvplot.line(
    x='transaction_date',
    y='transaction_amount',
    xlabel='Transaction Date',
    ylabel='Transaction Amount',
    title = 'Time Series of Transactions for Cardholder 18'
)

cardholder_eighteen_plot

:Curve   [transaction_date]   (transaction_amount)

In [ ]:
# Combined plot for card holders 2 and 18
important_customer_combined_plot = cardholder_two_plot * cardholder_eighteen_plot
important_customer_combined_plot.opts(
    title  = 'Time Series of Transactions for Cardholder 2 and 18'
)
important_customer_combined_plot


:Overlay
   .Curve.I  :Curve   [transaction_date]   (transaction_amount)
   .Curve.II :Curve   [transaction_date]   (transaction_amount)

Cardholder eighteen has 11 transactions that are abnormal from their normal spending. The transactions are high in value to trigger an alert and investigate. They occur about once a month which could be a reoccuring expense, but checking in with the customer is recommended.  

## Data Analysis Question 4

The CEO of the biggest customer of the firm suspects that someone has used her corporate credit card without authorization in the first quarter of 2018 to pay quite expensive restaurant bills. Again, for privacy reasons, you know only that the cardholder ID in question is 25.

* Using hvPlot, create a box plot, representing the expenditure data from January 2018 to June 2018 for cardholder ID 25.

* Are there any outliers for cardholder ID 25? How many outliers are there per month?

* Do you notice any anomalies? Describe your observations and conclusions in your markdown report.

In [ ]:
# loading data of daily transactions from jan to jun 2018 for card holder 25
# Write the query
query = """
        SELECT  
                ch.id AS cardholder_id,
                t.id AS transaction_id,
                t.date AS transaction_date,
                t.amount AS transaction_amount 
        FROM card_holder ch 
        JOIN credit_card cc ON ch.id = cc.cardholder_id
        JOIN transaction t ON cc.card = t.card
        WHERE cc.cardholder_id IN (25) AND t.date>='2018-01-01' AND t.date<'2018-07-01'
        ORDER BY t.date
        """
# Create a DataFrame from the query result. HINT: Use pd.read_sql(query, engine)
ceo_df = pd.read_sql(query, engine)

# View DataFrame
ceo_df


,cardholder_id,transaction_id,transaction_date,transaction_amount
0,25,2083,2018-01-02 02:06:21,1.46
1,25,1552,2018-01-05 06:26:45,10.74
2,25,2108,2018-01-07 14:57:23,2.93
3,25,754,2018-01-10 00:25:40,1.39
4,25,3023,2018-01-14 05:02:22,17.84
...,...,...,...,...
63,25,2582,2018-06-22 06:16:50,1813.00
64,25,3218,2018-06-23 22:36:00,16.61
65,25,1523,2018-06-25 09:17:05,11.53
66,25,2264,2018-06-27 14:33:06,5.24


In [ ]:
# Creating the six box plots using hvPlot
ceo_df['month'] = ceo_df['transaction_date'].dt.month_name()

ceo_plot = ceo_df.hvplot.box(
    y='transaction_amount',
    by='month',
    xlabel='Month',
    ylabel='Transaction Amount',
    title = 'Expenditure Jan 2018 - Jun 2018'
)

ceo_plot

:BoxWhisker   [month]   (transaction_amount)

There are several outliers. 
 - Jan - 1
 - Feb - 0
 - Mar - 1
 - Apr - 3
 - May - 1
 - Jun - 3


Since the CEO was concerned about fraudulant transactions in the first quarter of 2018. There are two transactions that stand out. One in Jan and the other in March. 